# Contoh Model Dense PyTorch untuk Klasifikasi MNIST

Import library yang akan digunakan seperti berikut

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

Untuk contoh ini, kita akan menggunakan dataset terkenal MNIST

MNIST merupakan dataset klasik yang merepresentasikan angka dalam tulisan tangan

In [ ]:
# Download dan load dataset training
train_dataset = datasets.MNIST('data', train=True, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Download dan load dataset test
test_dataset = datasets.MNIST('data', train=False, transform=transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

Lalu kita rancang arsitektur modelnya dengan 3 dense layer

In [ ]:
class SimpleDenseModel(nn.Module):
    def __init__(self):
        super(SimpleDenseModel, self).__init__()
        # Input: 28x28 = 784 pixels
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(784, 128)
        self.dense2 = nn.Linear(128, 64)
        self.dense3 = nn.Linear(64, 10)  # 10 kelas (digit 0-9)
    
    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.dense1(x))
        x = F.relu(self.dense2(x))
        x = self.dense3(x)
        return F.log_softmax(x, dim=1)

# Inisialisasi model
model = SimpleDenseModel()
# Pindahkan ke GPU jika tersedia
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Definisikan optimizer dan loss function
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

Selanjutnya kita buat juga fungsi untuk training dan testing modelnya

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # Pindahkan data ke device (CPU/GPU)
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad() # Reset gradient
        output = model(data) # Forward pass
        loss = criterion(output, target) # Hitung loss
        loss.backward() # Backward pass
        optimizer.step() # Update weights
        
        # Print progress
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():  # Tidak perlu menghitung gradient untuk testing
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    print(f'\nTest set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({accuracy:.2f}%)\n')
    
    return accuracy

Panggil kedua fungsi tersebut

In [ ]:
# List untuk menyimpan metrics
accuracies = []

# Training loop
n_epochs = 4
for epoch in range(1, n_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)
    accuracies.append(accuracy)

Dan akhirnya simpan model yang sudah kita train

In [ ]:
torch.save(model.state_dict(), 'mnist_model.pth')
print("Model berhasil disimpan sebagai 'mnist_model.pth'")